In [1]:
from preprocessing import *
import pickle
from joblib import load

In [5]:
# train_data
df_train, X_train, X_valid, y_train, y_valid = preprocessing(test_size=0.3, file_path='/Users/woojinheo/Desktop/github/SHAP/data/train.csv')

# test_data
df_test, X_test, _, y_test, _ = preprocessing(test_size=0, file_path='/Users/woojinheo/Desktop/github/SHAP/data/test.csv')

# load model

In [15]:
# logistic_regression
# logistic_model = load('/Users/woojinheo/Desktop/github/SHAP/models/logistic_regression_model.pkl')

In [ ]:
# # logistic_regression
# logistic_model = load('/Users/woojinheo/Desktop/github/SHAP/models/logistic_regression_model.pkl')

# decision_tree
with open('/Users/woojinheo/Desktop/github/SHAP/models/decision_tree_model.pkl', 'rb') as file:
    dt_model = pickle.load(file)

# catboost
with open('/Users/woojinheo/Desktop/github/SHAP/models/catboost_model.pkl', 'rb') as file:
    catboost_model = pickle.load(file)

# # random_forest
# rf_model = load('/Users/woojinheo/Desktop/github/SHAP/models/random_forest_model.pkl')

# predict (unseen data)

In [9]:
dt_pred = dt_model.predict(X_test)

In [11]:
cat_pred = catboost_model.predict(X_test)

In [22]:
print('decision_tree: ',accuracy_score(y_test,dt_pred))
print('catboost: ',accuracy_score(y_test,cat_pred))

decision_tree:  0.9047967354481059
catboost:  0.9596165691407453


# compare

In [13]:
df_test['dt_pred'] = dt_pred
df_test['cat_pred'] = cat_pred

In [24]:
check = pd.DataFrame(df_test[['satisfaction','dt_pred','cat_pred']].value_counts(normalize=True)).reset_index()

In [26]:
# both wrong
check[(check.satisfaction!=check.dt_pred)&(check.satisfaction!=check.cat_pred)]

,satisfaction,dt_pred,cat_pred,proportion
4,1,0,0,0.021019
5,0,1,1,0.010048


In [ ]:
# only cat_pred is wrong
check[(check.satisfaction==check.dt_pred)&(check.satisfaction!=check.cat_pred)]

,satisfaction,dt_pred,cat_pred,proportion
6,1,1,0,0.005659
7,0,0,1,0.003657


In [28]:
# only dt_pred is wrong
check[(check.satisfaction!=check.dt_pred)&(check.satisfaction==check.cat_pred)]

,satisfaction,dt_pred,cat_pred,proportion
2,1,0,1,0.032800
3,0,1,0,0.031337
